In [37]:
import pandas as pd
import numpy as np

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from scipy.special import softmax

from dense_layer import DenseLayer
from relu_layer import ReluLayer

In [20]:
iris = load_iris()
X, y = iris.data, iris.target

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, random_state=42, stratify=y, test_size=0.25)

In [38]:
def calculate_grads(X, y):
    probs = softmax(X)
    n = X.shape[0]
    probs[range(n), y] -= 1
    return probs / n

In [27]:
network = []
network.append(DenseLayer(X_train.shape[1], 40))
network.append(ReluLayer())
network.append(DenseLayer(40, 20))
network.append(ReluLayer())
network.append(DenseLayer(20, 3))

In [39]:
def forward(X):
    activations = []
    entr = X
    
    for layer in network:
        activations.append(layer.forward(entr))
        entr = activations[-1]
    
    return activations

In [ ]:
def predict(X):
    acts = softmax(forward(X))
    retur